<a href="https://colab.research.google.com/github/Tiger-A/GoogleCollab/blob/main/Copy_of_%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_26_%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8F_%D0%94%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_chatGPT_%D0%94%D0%B5%D0%BD%D1%8C_2_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6%D0%BD%D0%B8%D0%BA%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек. Сервисные функции
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb
!pip -q install gspread oauth2client

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class GPT():
  def __init__(self):
    pass

  @classmethod
  def set_key(cls):
      password_input = widgets.Password(
          description='Введите пароль:', 
          layout=widgets.Layout(width='500px'),
          style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
      login_button = widgets.Button(description='Авторизация')
      output = widgets.Output()

      def on_button_clicked(_):
          with output:
              openai.api_key = password_input.value
              os.environ["OPENAI_API_KEY"] = openai.api_key
              print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
              password_input.layout.display = 'none'
              login_button.layout.display = 'none'

      login_button.on_click(on_button_clicked)
      display(widgets.VBox([password_input, login_button, output]))

  def load_search_indexes(self, url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return self.create_embedding(text)

  def load_prompt(self, 
                  url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


  def create_embedding(self, data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
      source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

  def answer(self, system, topic, temp = 1):    
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

  def num_tokens_from_messages(self, messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
        
  def insert_newlines(self, text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


  def dialog(self):
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
      user = input('Клиент: ')
      if (user == 'Stop'): break

      dialog += '\n\n' + 'Клиент: ' + user
      add_dialog = gpt.answer(expert_prompt, user)
        
      dialog += '\n\n' + 'Менеджер: ' + add_dialog      
      print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {self.insert_newlines(add_dialog)}')
      report = gpt.answer(validation_prompt, dialog)
      answer = gpt.answer(action_prompt, report)

      print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
      print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{self.insert_newlines(answer)}\n\n')

    return dialog

  def answer_index(self, system, topic, search_index, temp = 1, verbose = 0):
    
    #Выборка документов по схожести с вопросом 
    docs = search_index.similarity_search(topic, k=5)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{self.num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

    # return completion

  def get_chatgpt_ansver3(self, system, topic, search_index, temp = 1):
    
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

In [ ]:
gpt = GPT()

In [ ]:
GPT.set_key()

<h3><b>Модель экспертности</b></h3>

In [ ]:
expert_promt = gpt.load_prompt('https://docs.google.com/document/d/181Q-jJpSpV0PGnGnx45zQTHlHSQxXvkpuqlKmVlHDvU/edit?usp=sharing')

In [ ]:
answer = gpt.answer(expert_promt, 
           'Я хочу, чтобы после завершения обучения мне подобрали работу')
print(answer)

Мы с удовольствием поможем вам найти работу после завершения обучения. У нас есть гарантия трудоустройства по договору. Мы уже трудоустроили большое количество наших студентов и имеем десятки компаний-партнеров, которые просят нас подобрать для них наших выпускников. Также у нас есть стажировки в крупных компаниях, которые являются очень сильной строчкой в резюме наших студентов. Кроме того, у нас есть собственное AI HR агентство AI-hunter, которое специализируется только на трудоустройстве AI разработчиков.


In [ ]:
answer = gpt.answer(expert_promt, 
           'Я хочу, чтобы после завершения обучения мне подобрали работу')
print(answer)

Конечно! Мы можем предложить вам гарантию трудоустройства по договору. Мы уже трудоустроили сотни студентов, у нас есть десятки компаний партнеров, которые просят нас подбирать их наших выпускников. Также у нас есть стажировки в крупных компаниях, которые являются очень сильной строчкой в резюме. И у нас есть внутреннее агентство - AI-hunter, которое специализируется на трудоустройстве AI-разработчиков. Мы готовы помочь найти вам работу после курсов!


In [ ]:
answer = gpt.answer(expert_promt, 
           'Мне надо создать AI проект на работе')
print(answer)

Мы рады, что вы заинтересованы в создании AI проекта на работе! Наш курс включает в себя поддержку в создании AI проекта, поэтому мы можем помочь вам в этом. Стоимость обучения составляет 219 900 рублей, однако мы предоставляем скидку в 30%, если вы зарегистрируетесь в течение двух дней, и обучение начнется с нуля без необходимости иметь опыт в программировании или математике. Оцените, пожалуйста, это предложение и скажите, будем ли мы работать вместе?


In [ ]:
answer = gpt.answer(expert_promt, 
           'Я бы хотел, но у меня нет 150 тысяч на обучение')
print(answer)

Понимаю вашу ситуацию, и хотел бы предложить вам несколько вариантов решения этой проблемы. 

1. Мы можем предоставить рассрочку от банков на 12, 24 и 36 месяцев, которая значительно снизит ежемесячный платеж. 

2. Мы также предлагаем внутреннюю рассрочку на 3 месяца без процентов. 

3. Помимо этого, мы можем гарантировать, что вы окупите обучение всего через 8 месяцев после трудоустройства. 

4. Наконец, вы также можете взять на себя опцию выкупа стоимости обучения и получить всю стоимость обучения обратно в виде зарплаты, работая у нас. 

Надеюсь, что с одним из этих вариантов вы сможете найти подходящую для вас опцию.


In [ ]:
answer = gpt.answer(expert_promt, 
           'У меня сейчас очень много работы, не уверен, что буду успевать обучаться')
print(answer)

Мы полностью понимаем, что у наших клиентов может быть много дел и обязанностей, поэтому мы предлагаем гибкий формат обучения, где учиться можно в удобное для вас время. На обучение всего лишь нужно в неделю 10 часов, а также всегда можно сделать заморозку обучения. Большинство наших студентов - очень занятые люди, но все находят время на обучение. Вы также можете выбрать продолжительность обучения в зависимости от своей занятости: 12, 24 или 36 месяцев.


In [ ]:
answer = gpt.answer(expert_promt, 
           'Сколько стоит обучение?')
print(answer)

Стоимость обучения составляет 219 900 рублей. Однако, в течение двух дней Вы можете получить скидку 30% и приобрести курс за 149 900 рублей. В состав курса входят: 3 года и 8 месяцев обучения, гарантия трудоустройства, поддержка в создании AI проекта и 4 стажировки на реальных AI проектах.


In [ ]:
answer = gpt.answer(expert_promt, 
           'У меня совсем нет опыта в программировании')
print(answer)

Не беспокойтесь, наш курс по AI разработке начинается с самых основ, обучение проходит с нуля, без опыта программирования и математики. Более того, в процессе обучения вам будут доступны все необходимые материалы и инструменты, чтобы успешно освоить AI разработку. Более 80% наших студентов раньше не имели опыта в программировании, но нашли общий язык с нашими кураторами и стали успешными AI разработчиками. Если вы заинтересовались, у нас есть гибкий формат обучения и мы можем провести вам пробное занятие, чтобы вы смогли понять, подходит ли вам наш курс.


Создание диалога

In [ ]:
def run_dialog():
  user = ''
  dialog = ''
  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break
    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
    
    print('\n')
    print('Менеджер: ' + add_dialog)
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('\n')

  return dialog

In [ ]:
dialog = run_dialog()

Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться


Менеджер: Мы полностью понимаем ваше занятость. У нас гибкий формат обучения, благодаря которому вы можете учиться в удобное для вас время. На обучение нужно всего 10 часов в неделю, и вы можете выбирать удобный график занятий. Кроме того, у нас всегда можно сделать заморозку обучения, если у вас возникнут временные трудности. Мы готовы поддержать вас в учебе и помочь вам достичь поставленных целей.


Клиент: Мне надо создать AI проект на работе


Менеджер: Здравствуйте! Рада, что вы обратились к нам за помощью. Наш курс по AI представляет из себя идеальное решение для тех, кто хочет создать свой AI проект и применить его в своей компании или профессии. В нашу программу обучения входит обучение всех основ AI технологий и создание различных AI проектов, которые помогут вам на практике. Более того, у нас есть возможность составления индивидуальной учебной программы, а также предоставление консультаций с ку

In [ ]:
print(dialog)


Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться
Менеджер: Мы полностью понимаем ваше занятость. У нас гибкий формат обучения, благодаря которому вы можете учиться в удобное для вас время. На обучение нужно всего 10 часов в неделю, и вы можете выбирать удобный график занятий. Кроме того, у нас всегда можно сделать заморозку обучения, если у вас возникнут временные трудности. Мы готовы поддержать вас в учебе и помочь вам достичь поставленных целей.
Клиент: Мне надо создать AI проект на работе
Менеджер: Здравствуйте! Рада, что вы обратились к нам за помощью. Наш курс по AI представляет из себя идеальное решение для тех, кто хочет создать свой AI проект и применить его в своей компании или профессии. В нашу программу обучения входит обучение всех основ AI технологий и создание различных AI проектов, которые помогут вам на практике. Более того, у нас есть возможность составления индивидуальной учебной программы, а также предоставление консультаций с куратор

In [ ]:
dialog_new = run_dialog()

Клиент: А сколько стоит курс?


Менеджер: Стоимость курса составляет 219 900 рублей, однако, если вы приобретете его в течение двух дней, мы можем предложить скидку в 30% и продать курс за 149 900 рублей. В состав курса входят 3 года и 8 месяцев обучения, гарантия трудоустройства, поддержка в создании AI проекта и 4 стажировки на реальных AI проектах.


Клиент: Stop


In [ ]:
dialog_new_2 = run_dialog()

Клиент: Я бы хотел трудоустроиться после обучения


Менеджер: Отлично, мы можем предложить Вам гарантию трудоустройства по договору. Уже многие наши студенты трудоустроились в качестве AI разработчиков. Также у нас есть успешные стажировки в крупных компаниях, которые являются очень сильной строчкой в резюме. Кроме того, мы имеем десятки компаний-партнеров, которые просят нас подобрать им наших выпускников. И посредством нашего внутреннего AI HR агентства AI-hunter, мы поможем Вам найти подходящее место работы. Вы можете быть уверены, что наш курс подготовит Вас к карьере AI разработчика и поможет с трудоустройством.


Клиент: Я бы хотел, но мне дорого по стоимости


Менеджер: Понимаю ваше возражение по стоимости, но хочу вам рассказать о нескольких вариантах, которые могут сделать обучение более доступным.

Во-первых, мы можем предоставить рассрочку от банков на 12, 24 и 36 месяцев, что поможет вам распределить стоимость обучения на более длительный период времени.

Во-вторых, у нас е

<h3><b>Модель оценки</b></h3>

In [ ]:
validation_promt = gpt.load_prompt('https://docs.google.com/document/d/1deHxH4rTpuJLJ0fnvsWJe8RwFbpju0-hVLLqklnlAL4/edit?usp=sharing')

In [ ]:
validation = gpt.answer(validation_promt, dialog_new)
print(validation)

1. Потребность о трудоустройстве - не было
2. Потребность о создании AI проекта - не было
3. Возражение о стоимости - было
4. Возражение о времени обучения - не было
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - не было


In [ ]:
validation = gpt.answer(validation_promt, dialog)
print(validation)

1. Потребность о трудоустройстве - не была
2. Потребность о создании AI проекта - была
3. Возражение о стоимости - было
4. Возражение о времени обучения - не было
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - не был


In [ ]:
validation_promt_full = gpt.load_prompt('https://docs.google.com/document/d/1cRtbTSEy7F6Thd1utbb7XJoUbSeg_bhegbWLoosWLLc/edit?usp=sharing')

In [ ]:
validation = gpt.answer(validation_promt_full, dialog)
print(validation)

1. Потребность о трудоустройстве - не была
2. Потребность о создании AI проекте - была
3. Возражение о стоимости - было
4. Возражение о времени обучения - было
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - не был


In [ ]:
validation = gpt.answer(validation_promt_full, dialog_new_2)
print(validation)

1. Потребность о трудоустройстве - была
2. Потребность о создании AI проекта - не была
3. Возражение о стоимости - было
4. Возражение о времени обучения - не было
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - не был


In [ ]:
validation = gpt.answer(validation_promt_full, dialog_new_2, temp = 0.1)
print(validation)

1. Потребность о трудоустройстве - была
2. Потребность о создании AI проекта - нет
3. Возражение о стоимости - было
4. Возражение о времени обучения - нет
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - нет


In [ ]:
validation = gpt.answer(validation_promt_full, dialog_new_2, temp = 0.1)
print(validation)

1. Потребность о трудоустройстве - была
2. Потребность о создании AI проекта - нет
3. Возражение о стоимости - было
4. Возражение о времени обучения - нет
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - нет


<h3><b> Модель действия </h3>

In [ ]:
action_promt = gpt.load_prompt('https://docs.google.com/document/d/1lZNKuFWtyO-m7r8E2bXoDTAY4SzpR2ylMgz33RVU0oI/edit')

In [ ]:
question = gpt.answer(action_promt, validation)
print(question)

"Актуально ли вам создать AI проект в процессе обучения? Если да, то какой именно проект вы рассмотрели бы? Кстати, мы предлагаем не только обучение, но и возможность стажировки у наших партнеров. Это поможет вам получить опыт работы над реальными проектами и улучшить свои навыки. Что касается стоимости, мы предлагаем различные варианты оплаты и скидки для наших студентов. Хотели бы вы обсудить дополнительные возможности и условия обучения?"


In [ ]:
question = gpt.answer(action_promt_test, validation, temp = 0.1)
print(question)

"Понимаю, что стоимость обучения может быть важным фактором. Но если говорить про трудоустройство, вас интересует трудоустройство после обучения?"


In [ ]:
action_promt_test = gpt.load_prompt('https://docs.google.com/document/d/1mX-pXzuxkygAqnwRGfrISAx99J7VnVvPg4EYAzNFdsk/edit?usp=sharing')

In [ ]:
question = gpt.answer(action_promt_test, validation, temp = 0.1)
print(question)

Здравствуйте! Я получил отчет о вашем диалоге с нашим менеджером по продажам. Я вижу, что у вас есть потребность в трудоустройстве, но не было потребности в создании AI проекта. А не хотите ли вы создать свой AI проект? Касательно возражения о стоимости, я могу сказать, что наши курсы по AI являются одними из самых конкурентоспособных на рынке, и мы предлагаем различные варианты оплаты. Если у вас есть какие-то вопросы по стоимости обучения, я готов на них ответить. А по поводу опыта, наши курсы доступны как для начинающих, так и для профессионалов в области AI. Если у вас есть какие-то вопросы, я готов на них ответить. Как я могу вам помочь?


In [ ]:
question = gpt.answer(action_promt_test, validation, temp = 0.1)
print(question)

Спасибо за информацию о вашей потребности в трудоустройстве. Я хотел бы уточнить, какие именно навыки в AI вы считаете наиболее важными для вашей будущей карьеры?


<h3><b> Полный диалог </h3>

In [ ]:
def run_full_dialog():
  user = ''
  dialog = ''

  print('С чем связан ваш интерес к искусственному интеллекту?')

  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break

    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
      
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('Менеджер: ' + add_dialog)
    report = gpt.answer(validation_promt_full, dialog)
    answer = gpt.answer(action_promt, report)

    print('Отчёт системы:\n' + report)
    print('\n')
    print('Менеджер 2: ' + answer)
    print('\n')

  return dialog

У меня сейчас очень много работы, не уверен, что буду успевать обучаться

Мне надо создать AI проект на работе

Я бы хотел трудоустроиться после завершения обучения

In [ ]:
dialog = run_full_dialog()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться


Менеджер: Наш курс обучения имеет гибкий формат обучения, который позволяет учиться в любое время, когда на это есть свободное время. Также у нас всегда есть возможность заморозить обучение и вернуться к нему позже, если Вам будет нужно. На обучение нужно всего 10 часов в неделю, что позволит Вам уделить время работе и другим делам.


Отчёт системы:
1. Потребности в трудоустройстве - нет
2. Потребности создания AI проекта - нет
3. Возражения о стоимости - не было
4. Возражения о времени обучения - было
5. Вопрос про стоимость обучения - не был
6. Вопрос про опыт - не был


Менеджер: "Уточните, пожалуйста, какие именно вопросы у вас возникли по поводу времени обучения. Мы можем предложить вам гибкий график обучения, который будет составлен в соответствии с вашими задачами и доступностью времени."


Клиент: Мне надо создать AI проект на работе


Менедже

In [ ]:
action_promt_2 = gpt.load_prompt('https://docs.google.com/document/d/1U2ZeL2TAjGzIY5aOGnhePwZFx0dSZQzSGOLEd0k7DhU/edit')

In [ ]:
def run_full_dialog_2():
  user = ''
  dialog = ''

  print('С чем связан ваш интерес к искусственному интеллекту?')

  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break

    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
      
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('Менеджер: ' + add_dialog)
    report = gpt.answer(validation_promt_full, dialog)
    answer = gpt.answer(action_promt_2, report)

    print('\nОтчёт системы:\n' + report)
    print('\n')
    print('Менеджер 2: ' + answer)
    print('\n')

  return dialog

У меня сейчас очень много работы, не уверен, что буду успевать обучаться

Мне надо создать AI проект на работе

Я бы хотел трудоустроиться после завершения обучения

In [ ]:
dialog = run_full_dialog_2()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться
Менеджер: Понимаю, что у вас много дел, но наш курс подойдет для занятых людей. Мы предоставляем гибкий формат обучения, учиться можно в любое время, а проходит он всего 10 часов в неделю. Кроме того, в любой момент вы можете сделать заморозку обучения, если у вас возникнут неотложные дела. К тому же, у нас есть гарантия трудоустройства после окончания обучения, так что вы точно окупите потраченное время и сможете применять полученные знания на практике.

Отчёт системы:
1. Потребность о трудоустройстве - была или нет: Была
2. Потребность о создании AI проекте - не была
3. Возражение о стоимости - не было
4. Возражение о времени обучения - было
5. Вопрос про стоимость обучения - не был
6. Вопрос про опыт - не был


Менеджер 2: Актуально ли вам трудоустройство после обучения? При этом обучение позволит пройти путь от начального уровня до работы на реальны

Третья версия

In [ ]:
action_promt_3 = gpt.load_prompt('https://docs.google.com/document/d/1Cy3_aUdu4tLknZDMCslm-Y85MJQNGiN5b5X_d8WQ6kU/edit?usp=sharing')

In [ ]:
validation_promt_3 = gpt.load_prompt('https://docs.google.com/document/d/1DBpjbpGt9daCC302lkT079xCHZKdyexlcQazafLCRVs/edit?usp=sharing')

In [ ]:
def run_full_dialog_3():
  user = ''
  dialog = ''

  print('С чем связан ваш интерес к искусственному интеллекту?')

  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break

    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
      
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('Менеджер: ' + add_dialog)
    report = gpt.answer(validation_promt_3, dialog, temp = 0.1)
    answer = gpt.answer(action_promt_3, report, temp = 0.1)

    print('\nОтчёт системы:\n' + report)
    print('\n')
    print('Менеджер 2: ' + answer)
    print('\n')

  return dialog

У меня сейчас очень много работы, не уверен, что буду успевать обучаться

Мне надо создать AI проект на работе

Я бы хотел трудоустроиться после завершения обучения

In [ ]:
dialog = run_full_dialog_3()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться
Менеджер: Мы понимаем, что у вас много работы, но у нас гибкий формат обучения, поэтому учиться можно в любое время и с любого места. На обучение нужно всего 10 часов в неделю, поэтому вы всегда найдете время на обучение. Кроме того, мы даем возможность заморозить обучение в случае непредвиденных обстоятельств. Как только вы закончите обучение, мы гарантируем трудоустройство по договору, поэтому вы точно вернете вложения в обучение в виде новой работы.

Отчёт системы:
1. Потребность о трудоустройстве - есть
2. Потребность о создании AI проекта - нет
3. Возражение о стоимости - отсутствует
4. Возражение о времени обучения - есть
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - отсутствует


Менеджер 2: Актуально ли вам трудоустройство после обучения? Если да, то предлагаю рассмотреть наши курсы, которые помогут вам получить необходимые знания 

In [ ]:
action_promt_4 = gpt.load_prompt('https://docs.google.com/document/d/15PjoEh_s4iHeWq5s5B79Y-Bn1BgWHrUzPrgTayNxOBw/edit?usp=sharing')

In [ ]:
def run_full_dialog_4():
  user = ''
  dialog = ''

  print('С чем связан ваш интерес к искусственному интеллекту?')

  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break

    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
      
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('Менеджер: ' + add_dialog)
    report = gpt.answer(validation_promt_3, dialog, temp = 0.1)
    answer = gpt.answer(action_promt_4, report, temp = 0.1)

    print('\nОтчёт системы:\n' + report)
    print('\n')
    print('Менеджер 2: ' + answer)
    print('\n')

  return dialog

У меня сейчас очень много работы, не уверен, что буду успевать обучаться

Мне надо создать AI проект на работе

Я бы хотел трудоустроиться после завершения обучения

In [ ]:
run_full_dialog_4()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться
Менеджер: Понимаю, что у вас много дел, но наше обучение является очень гибким и позволяет учиться в удобное для вас время. Даже если вы будете заниматься всего 10 часов в неделю, вы сможете успешно завершить обучение за 3 года и 8 месяцев. К тому же, мы предоставляем возможность замораживать обучение, если вам потребуется отложить занятия на некоторое время.

Отчёт системы:
1. Потребность о трудоустройстве - отсутствует
2. Потребность о создании AI проекта - отсутствует
3. Возражение о стоимости - отсутствует
4. Возражение о времени обучения - было
5. Вопрос про стоимость обучения - был
6. Вопрос про опыт - отсутствует


Менеджер 2: Здравствуйте! Спасибо за обращение. Я хотел бы уточнить, какие цели вы преследуете в области AI? Может быть, у вас есть какие-то проекты, которые вы хотели бы реализовать с помощью искусственного интеллекта?


Клиент: Мне н

'\nКлиент: У меня сейчас очень много работы, не уверен, что буду успевать обучаться\nМенеджер: Понимаю, что у вас много дел, но наше обучение является очень гибким и позволяет учиться в удобное для вас время. Даже если вы будете заниматься всего 10 часов в неделю, вы сможете успешно завершить обучение за 3 года и 8 месяцев. К тому же, мы предоставляем возможность замораживать обучение, если вам потребуется отложить занятия на некоторое время.\nКлиент: Мне надо создать AI проект на работе\nМенеджер: Здравствуйте! Мы можем предложить вам курс по AI, в рамках которого вы сможете создать свой AI проект и применить его в своей компании или профессии. Стоимость курса составляет 219.900 рублей, но в течение двух дней можно получить скидку 30% и приобрести курс за 149.900 рублей. Кроме того, мы предоставляем гарантию трудоустройства и поддержку в создании AI проекта. Обучение полностью с нуля, без опыта программирования и математики, и 80% наших студентов - никогда в прошлом не программировали

In [ ]:
validation_promt_5 = gpt.load_prompt('https://docs.google.com/document/d/10XEdAWYQkyZhQ67Px8OCRoDRYOnjsq7WJxCD4uMb8kg/edit?usp=sharing')

In [ ]:
action_promt_5 = gpt.load_prompt('https://docs.google.com/document/d/1gMPi0R1tZMtCUn3PKnDBmGSE2QseImT8A0x3jghgYwY/edit?usp=sharing')

In [ ]:
def run_full_dialog_5():
  user = ''
  dialog = ''

  print('С чем связан ваш интерес к искусственному интеллекту?')

  while user != 'Stop':
    user = input('Клиент: ')
    if (user == 'Stop'): break

    dialog += '\n' + 'Клиент: ' + user
    add_dialog = gpt.answer(expert_promt, user)
      
    dialog += '\n' + 'Менеджер: ' + add_dialog
    print('Менеджер: ' + add_dialog)
    report = gpt.answer(validation_promt_5, dialog)
    answer = gpt.answer(action_promt_5, report)

    print('\nОтчёт системы:\n' + report)
    print('\n')
    print('Менеджер 2: ' + answer)
    print('\n')

  return dialog

In [ ]:
run_full_dialog_5()

С чем связан ваш интерес к искусственному интеллекту?
Клиент: Я хотел бы создать свой проект по AI и применить его в компании
Менеджер: Прекрасно! Наш курс действительно поможет вам достичь этой цели. Сотни наших студентов уже создали свои собственные проекты, и мы поможем вам сделать то же самое. Мы можем составить для вас индивидуальную учебную программу, подобрать проект и помочь в его создании. Вам также будет доступна поддержка на каждом этапе. У нас есть возможность предоставить вам стажировку на реальных AI проектах, чтобы вы могли получить опыт работы с различными инструментами и технологиями. Если у вас есть какие-либо вопросы, я готов на них ответить.

Отчёт системы:
Потребности клиента:
- Развитие навыков в области AI
- Создание своего проекта по AI в компании
- Получение индивидуальной учебной программы
- Поддержка на каждом этапе создания проекта
- Получение опыта работы с различными инструментами и технологиями AI

Возражения клиента не были названы.

Менеджер предоставил

'\nКлиент: Я хотел бы создать свой проект по AI и применить его в компании\nМенеджер: Прекрасно! Наш курс действительно поможет вам достичь этой цели. Сотни наших студентов уже создали свои собственные проекты, и мы поможем вам сделать то же самое. Мы можем составить для вас индивидуальную учебную программу, подобрать проект и помочь в его создании. Вам также будет доступна поддержка на каждом этапе. У нас есть возможность предоставить вам стажировку на реальных AI проектах, чтобы вы могли получить опыт работы с различными инструментами и технологиями. Если у вас есть какие-либо вопросы, я готов на них ответить.\nКлиент: Компании достаточно важно, но я не уверен, что мне хватит времени на обучение и на создание проекта\nМенеджер: Понимаю вашу ситуацию и сомнения. Хочу вам сообщить, что мы имеем гибкий формат обучения, поэтому учиться можно, когда угодно, чтобы вам было удобно и не нужно было жертвовать своими обязательствами. Кроме того, требуется лишь 10 часов в неделю на обучение, та